In [1]:
import pandas as pd
from signalai.config import CONFIG_DIR, PIPELINE_SAVE_PATH
from taskorganizer.pipeline import Pipeline
from signalai.signal_tools.signal import SignalDataset, Signal

import torch 
import torch.nn as nn
import torch.nn.functional as F
from inceptiontime import InceptionBlock
import torch.optim as optim
from tqdm import tqdm, trange
import numpy as np

%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'svg'

In [2]:
config_path = CONFIG_DIR / "processing" / "pipeline.yaml"
params_config_path = CONFIG_DIR / "data_preparation" / "diamond_cropped.yaml"
generator_config_path = CONFIG_DIR / "generators" / "diamond.yaml"
pip = Pipeline(
    config_path,
    config_dir=CONFIG_DIR,
    save_folder=PIPELINE_SAVE_PATH,
    parameter_yamls=[params_config_path, generator_config_path]
)

In [3]:
# df = pip.run("DatasetLoader")

In [6]:
gen_gen = pip.run("data_generator")
gen = gen_gen.get_generator("train", log=0, batch_size=16)

Loading datasets ['diamond_loc00', 'diamond_loc01', 'diamond_loc02', 'diamond_loc03', 'diamond_loc04', 'diamond_loc05', 'diamond_loc06', 'diamond_loc07', 'diamond_loc08', 'diamond_loc09'] to RAM: 100%|██████████| 2000/2000 [00:49<00:00, 40.67it/s]


In [24]:
# gen_gen.signal_loader.loaded_signals["AEforANN-tube1-diamond-loc00_ch_001.bin-0-valid"]

                                                               0
dataset                                            diamond_loc00
filename_id      AEforANN-tube1-diamond-loc00_ch_001.bin-0-valid
split                                                      valid
interval_start                                              9500
interval_end                                               25884
interval_length                                            16384
values                                                    100000
frequency                                                1562500
big_endian                                                  True
source_dtype                                             float32
dtype_bytes                                                    4
signed                                                      True
op_dtype                                                 float32
to_ram                                                      True
standardize              

In [17]:
%%time
for i in tqdm(range(2000)):
    X, Y = next(gen)
    _=torch.from_numpy(np.array(X)).to("cuda")
    _=torch.from_numpy(np.array(Y)).to("cuda")

100%|██████████| 2000/2000 [00:05<00:00, 381.27it/s]

CPU times: user 5.22 s, sys: 27.4 ms, total: 5.25 s
Wall time: 5.25 s


In [12]:
class Net(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.inception_block1 = InceptionBlock(
            in_channels=1,
            n_filters=32,
            kernel_sizes=[11, 21, 41],
            bottleneck_channels=32,
            use_residual=True,
            activation=nn.SELU()
        )
        self.adaptive_pool = nn.AdaptiveAvgPool1d(output_size=1)
        self.linear1 = nn.Linear(in_features=4 * 32 * 1, out_features=11)
        self.out_activation = nn.Softmax(dim=-1)
        
    def forward(self, x):
        x = self.inception_block1(x)
        x = self.adaptive_pool(x)
        x = x.view(-1, 32 * 4 * 1)
        x = self.linear1(x)
        y = self.out_activation(x)
        return y

    
net = Net().to(device)

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)

In [14]:
echo_step = 400
batches_id = trange(16000)
for train_batch in batches_id:  # loop over the dataset multiple times
    X, Y = next(gen)
    inputs = torch.from_numpy(np.array(X)).to(device)
    labels = torch.from_numpy(np.array(Y)).to(device)
    # print(labels)
    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # print statistics
    running_loss += loss.item()
    batches_id.set_description(f"Loss: {loss.item(): .04f}")
    if train_batch % echo_step == 0:
        print()

print('Finished Training')

  0%|          | 0/4000 [00:00<?, ?it/s]/home/martin/anaconda3/envs/torch/lib/python3.8/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448234945/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)
Loss:  2.3968:   0%|          | 1/4000 [00:00<25:43,  2.59it/s]

Loss:  2.3483:  10%|█         | 401/4000 [01:34<14:04,  4.26it/s]

Loss:  2.2613:  20%|██        | 801/4000 [03:08<12:26,  4.29it/s]

Loss:  2.2127:  30%|███       | 1201/4000 [04:45<10:55,  4.27it/s]

Loss:  2.1360:  40%|████      | 1601/4000 [06:19<09:25,  4.25it/s]

...

Loss:  2.0274:  70%|███████   | 2801/4000 [11:02<04:37,  4.31it/s]

Loss:  1.8874:  80%|████████  | 3201/4000 [12:36<03:10,  4.18it/s]

Loss:  1.8857:  90%|█████████ | 3601/4000 [14:10<01:34,  4.24it/s]

Loss:  1.8550: 100%|██████████| 4000/4000 [15:44<00:00,  4.24it/s]

Finished Training


In [15]:
with torch.no_grad():
    inputs, labels = next(val_gen)
    inputs = torch.tensor(inputs, device="cuda", dtype=torch.float32).unsqueeze(0)
    labels = torch.tensor(labels, device="cuda", dtype=torch.long).unsqueeze(0)
    print(net(inputs))

NameError: name 'val_gen' is not defined

In [ ]:
s,_=next(gen)
s=Signal(s)

In [ ]:
128**2

In [ ]:
s.show()

In [ ]:
s.signal